In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
# Load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import pickle


%config Completer.use_jedi = False 




#--- Notebook run settings
run = 'command line' # 'manual' 'command line'
#run = 'manual'


# Settings for the multiple linear regression model (MLR)
#--- Arguments
if run == 'command line':
    # Read command line arguments
    arguments = os.environ['NB_ARGS']
    alpha, date_exp_str = arguments.split(',')
    
    alpha              = float(alpha)   
    
elif run == 'manual':
    # Enter arguments manually 
    alpha              = 0.01
    date_exp_str       = '230207_Full'
    remove_data        = True

# Mortality group?
if ('Full' in date_exp_str):
        
    print(date_exp_str, alpha)    

In [3]:
import glob

file_list = glob.glob('site_reg_results/'+date_exp_str+'_*'+str(alpha)+'*_SITE_best_Models_dAIC.pkl')
file_list = [f for f in file_list if 'False' not in f]
file_list

['site_reg_results/221114_Management_0.01_True_SITE_best_Models_dAIC.pkl',
 'site_reg_results/221114_Management_NoInsectsFire_0.01_True_SITE_best_Models_dAIC.pkl',
 'site_reg_results/221114_Management_FireOnly_0.01_True_SITE_best_Models_dAIC.pkl',
 'site_reg_results/221114_Management_NoFire_0.01_True_SITE_best_Models_dAIC.pkl']

In [4]:
#---------------#
#  Import data  #
#---------------#
mtf_type = ['count','cmass']
mortality_groups = ['M_All','M_Fire','M_NoFire','M_Other']


# Initialise storage
res_tables  = np.empty((len(mortality_groups),len(mtf_type)),dtype='object') # table; res_objects
df_res_mort = pd.DataFrame()
for f,file in enumerate(file_list):
    
    with open(file,'rb') as l:
        res_temp = pickle.load(l)
        
        for t in range(2):
            res_tables[f,t] = res_temp[t]

            
        res_tables_formatted = res_tables.copy()
        df_mort = pd.DataFrame()
        for t,tpe in enumerate(mtf_type):
            res_tables_formatted[f,t] = res_tables[f,t].set_index(['model','climate','Mortality','N_obs'])
            res_tables_formatted[f,t] = res_tables_formatted[f,t][['r2','AIC','d_AIC','RMSE','ID']].copy()
            res_tables_formatted[f,t].columns = pd.MultiIndex.from_tuples([(i,tpe) for i in 
                                                                           res_tables_formatted[f,t].columns])
        
        df_mort = pd.concat(res_tables_formatted[f], axis=1)
        df_mort = df_mort.reset_index(drop=False)
        df_mort = df_mort[['Mortality','model','r2','AIC','d_AIC','RMSE','ID','N_obs','climate']].copy()
        df_melt = df_mort.melt(ignore_index=False)
        idx_r2  = df_melt[df_melt.variable_0 == 'r2'].sort_values('value',ascending=False).index.unique()
        df_mort = df_mort.reindex(idx_r2).reset_index(drop=True).copy()
        df_mort.set_index(['Mortality','model'],inplace=True)
        df_res_mort = pd.concat([df_res_mort,df_mort],axis=0)

# Sort by mortality group
df_res_mort = df_res_mort.reset_index(drop=False)
df_res_mort.Mortality = df_res_mort.Mortality.astype("category")
df_res_mort.Mortality = df_res_mort.Mortality.cat.set_categories(mortality_groups)
df_res_mort['Level'] = 'Site' # Add analysis level
       
df_res_mort = df_res_mort.sort_values(["Mortality"]).set_index(['Level','Mortality','model'])
display(df_res_mort)
with open('site_reg_results/'+date_exp_str+'_'+str(alpha)+'_SITE_best_models_dAIC.pkl','wb') as f:
    pickle.dump(df_res_mort,f)

r2          AIC        d_AIC  \
                                         count cmass  count  cmass count   
Level Mortality model                                                      
Site  M_All     MTF ~ Temp + DBH_mean     0.33  0.35  143.5  153.4     0   
                MTF ~ MAsoilT + DBH_mean  0.26   0.3  148.9  158.2   5.4   
      M_Fire    MTF ~ DBH_mean            0.38   NaN   44.3    NaN     0   
      M_NoFire  MTF ~ Temp                0.37  0.41   96.1  103.2     0   
                MTF ~ MAsoilT             0.33  0.38   99.2  105.3   3.1   
                MTF ~ CDI                 0.28   NaN  101.9    NaN   5.8   
      M_Other   MTF ~ CDI                 0.41  0.43   82.3   89.1     0   
                MTF ~ Temp                0.35  0.41   85.8   90.6   3.5   
                MTF ~ MAsoilT             0.32  0.41   87.5   90.7   5.2   

                                                RMSE          ID       N_obs  \
                                         cmass count cmass count cmass         
Level Mortality model                                                          
Site  M_All     MTF ~ Temp + DBH_mean        0  24.3  51.7     0     2  61.0   
                MTF ~ MAsoilT + DBH_mean   4.8  26.4  55.2     5     4  61.0   
      M_Fire    MTF ~ DBH_mean             NaN  21.7   NaN     3   NaN  18.0   
      M_NoFire  MTF ~ Temp                   0  23.1  45.7     6     5  43.0   
                MTF ~ MAsoilT              2.1  24.6  48.9    12    13  43.0   
                MTF ~ CDI                  NaN  24.9   NaN    18   NaN  43.0   
      M_Other   MTF ~ CDI                    0  25.3    51     3     6  37.0   
                MTF ~ Temp                 1.5  24.5  48.4    14    14  37.0   
                MTF ~ MAsoilT              1.6  26.2    52    18    15  37.0   

                                            climate  
                                                     
Level Mortality model                                
Site  M_All     MTF ~ Temp + DBH_mean     CRUNCEPv7  
                MTF ~ MAsoilT + DBH_mean        NaN  
      M_Fire    MTF ~ DBH_mean                  NaN  
      M_NoFire  MTF ~ Temp                CRUNCEPv7  
                MTF ~ MAsoilT                   NaN  
                MTF ~ CDI                       NaN  
      M_Other   MTF ~ CDI                       NaN  
                MTF ~ Temp                CRUNCEPv7  
                MTF ~ MAsoilT                   NaN